In [ ]:
import numpy as np
from collections import Counter

def lea_datos(archivo, i_clase=4, encabezado=True, delim=","):
    '''Una funcion para leer archivos con datos de clasificación. 
    Argumentos:
        archivo - direccion del archivo
        i_clase - indice de columna que contiene las clases. 
                  default es -1 y significa la ultima fila.
        header - si hay un encabezado
        delim - que separa los datos
    Regresa:
        Un tuple de los datos, clases y cabezazo en caso que hay.'''
    
    todo = np.loadtxt(archivo, dtype="S", delimiter=delim)    # para csv
    if(encabezado):
        encabezado = todo[0,:]
        todo = todo[1:-2,:]
    else: 
        encabezado = None
    
    clases = todo[:, i_clase]
    datos = np.delete(todo, [0, i_clase], axis=1)
    print ("Clases")
    for k,v in Counter(clases).items(): print (k,":",v)
    return (datos, clases, encabezado) 

In [ ]:
datos, clases, encabezado = lea_datos("datos_peña.csv")  # _ significa que no nos interesa este valor 
clases.shape

In [ ]:
datos

In [ ]:
X = datos.astype("float")

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(oob_score=True)
clf.fit(X, clases)
print ("Exactitud estimado:", clf.oob_score_)

In [ ]:
%matplotlib inline
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt

cv = StratifiedKFold(y=clases, n_folds=2)
arboles_vals = np.arange(5,200,5)
busqueda = GridSearchCV(clf, param_grid=dict(n_estimators=arboles_vals), cv=cv)
busqueda.fit(X, clases)

print ('Mejor numero de arboles=',busqueda.best_params_,',exactitud =',busqueda.best_score_)

scores = [x[1] for x in busqueda.grid_scores_]

plt.plot(arboles_vals, scores)
plt.xlabel('C')
plt.ylabel('exactitud cv')
plt.show()

In [ ]:
prof_vals = np.arange(1,12)
busqueda = GridSearchCV(clf, param_grid=dict(max_depth=prof_vals), cv=cv)
busqueda.fit(datos, clases)

print ('Mejor profundidad=',busqueda.best_params_,',exactitud =',busqueda.best_score_)

scores = [x[1] for x in busqueda.grid_scores_]
plt.plot(prof_vals, scores)
plt.xlabel('profundidad maxima')
plt.ylabel('exactitud cv')
plt.show()

In [ ]:
#SVM
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score, StratifiedKFold

clf = SVC(kernel='rbf')
exas = cross_val_score(clf, X, clases, cv=2)    # escoge un numero adecuado de folds 
print ('cv exactitud =', exas.mean(), '±', exas.std())

In [ ]:
%matplotlib inline
from sklearn.grid_search import GridSearchCV
import matplotlib.pyplot as plt

cv = StratifiedKFold(y=clases, n_folds=3)     # usamos las primeras 100 muestras
C_rango = 10.0 ** np.arange(-5,5)
gamma_rango = 10.0 ** np.arange(-5,5)
busqueda = GridSearchCV(clf, param_grid=dict(C=C_rango, gamma=gamma_rango), cv=cv)
busqueda.fit(X, clases)        # usamos las primeras 100 muestras

print ('Mejor C, gamma =',busqueda.best_params_,',exactitud =',busqueda.best_score_)

scores = np.array( [x[1] for x in busqueda.grid_scores_] ) # Sacamos nada mas el segundo elemento de los scores
n_C, n_g = len(C_rango), len(gamma_rango)                  # sacamos el numero de parametros probados
scores = scores.reshape(n_C, n_g)                          # convertimos los scores en la cuadricula

# Ahora vamos a dibujar todo
plt.figure(figsize=(10,8))                     # hacemos el imagen un poco mas grande
plt.imshow(scores, interpolation='nearest')    # dibuja los scores representado por colores
plt.xlabel('gamma')                            # pon el titulo del eje x
plt.ylabel('C')                                # pon el titulo del eje y
plt.colorbar()                                 # pon un escala de colores para los scores
plt.xticks(np.arange(len(gamma_rango)), gamma_rango, rotation=45)    # pon los valores para gamma
plt.yticks(np.arange(len(C_rango)), C_rango)   # pon los valores para C
plt.show()